In [59]:
import pandas as pd
import numpy as np
data = pd.read_csv("binned_feature_matrix_and_labels_500bp/fetaures_and_labels.2_classes.500bp_bin.tsv", sep ='\t')

In [60]:
x_subset = data['chr'] == "chrX"
x_data = data.loc[x_subset]
x_data


,chr,start,stop,label,GSE39271_S2_GFPi_CLAMP,GSE39271_S2_MSL2i_CLAMP,GSE62904_CapH2,GSE62904_CP190,GSE62904_CTCF,GSE62904_Chromator,...,GSE62904_PcRJ,GSE62904_Z4,GSE62904_PcVP,GSE62904_TFIIIC,GSE62904_RNAPII,GSE62904_Rad21,GSE93100_Ash1,GSE93100_H3K27me3,GSE93100_H3K36me2,GSE93100_Mrg15
220679,chrX,0,500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
220680,chrX,500,1000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
220681,chrX,1000,1500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
220682,chrX,1500,2000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
220683,chrX,2000,2500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267759,chrX,23540000,23540500,0,14,12,25,5,40,25,...,19,22,8,38,0,21,10,0,0,3
267760,chrX,23540500,23541000,0,10,17,54,10,64,28,...,30,44,10,53,0,44,6,4,0,4
267761,chrX,23541000,23541500,0,33,70,56,14,73,25,...,23,71,11,62,2,50,6,7,0,9
267762,chrX,23541500,23542000,0,4,11,12,5,40,9,...,17,11,6,17,0,21,5,1,0,5


In [63]:
labels_subset = x_data['label'][10:-11]
x_data_nolabel = x_data.drop(columns=['chr', 'start', 'stop', 'label'])

1499


In [64]:
x_data_np = x_data_nolabel.to_numpy()
x_labels_np = x_data['label'].to_numpy()
np.save("mlp_data.np", x_data_np)
np.save("mlp_lbls.np", x_labels_np)

In [4]:
labels_subset.reset_index(drop = True)
print(labels_subset.shape, x_data_nolabel.shape)
# assert(labels_subset.shape[0] == x_data_nolabel.shape[0])

(47064,) (47085, 24)


In [5]:
import numpy as np
# Define the number of rows you want in each submatrix
num_rows_per_submatrix = 21
each_drxn = 10


# Extract 50 consecutive rows as submatrices
submatrices = np.stack([x_data_nolabel[i-each_drxn:i+each_drxn+1] for i in range(10, x_data_nolabel.shape[0]-each_drxn-1)])


In [6]:
### matrix of all data samples 
assert(labels_subset.shape[0] == submatrices.shape[0])
data = np.asarray(submatrices)
labels = np.asarray(labels_subset)
data.shape, labels.shape

((47064, 21, 24), (47064,))

In [40]:
## LABEL TRANSFORMATION -> sliding window 1s
# array([  214,   495,   502, ..., 46176, 46191, 46195])
binding_indices = np.where(labels == 1)[0]
positive_width = 5
new_labels = labels.copy()

for binding in binding_indices:
    start_indx = max(0, binding - positive_width)
    end_indx = min(len(labels), binding + positive_width + 1)
    new_labels[start_indx:end_indx] = 1

In [41]:
print(labels[210:219])
print(new_labels[210:219])

print(labels[491:507])
print(new_labels[491:507])

print(np.count_nonzero(new_labels == 1), np.count_nonzero(labels == 1))
print(np.count_nonzero(new_labels == 0), np.count_nonzero(labels == 0))

[0 0 0 0 1 0 0 0 0]
[1 1 1 1 1 1 1 1 1]
[0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
9056 1499
38008 45565


In [39]:
# Full Dataset Exporting
np.save("data/X_data.npy", data)
np.save("data/X_labels.npy", new_labels)

## Sample Small Toy Dataset to Run Locally for Model Verification

In [48]:
## Sample 3000 data points from the full dataset
import random
random.seed(123)

sampled_indices = random.sample(range(0, len(data)), 3000)
sampled_data = data[sampled_indices, :, :]
sampled_labels = new_labels[sampled_indices]

# Sampled Dataset Exporting
np.save("X_sampled_data.npy", sampled_data)
np.save("X_sampled_labels.npy", sampled_labels)

In [13]:
Counter(data['Chromosome'].to_list())

Counter({'2L': 76704,
         '2LHet': 992,
         '2R': 70488,
         '2RHet': 9048,
         '3L': 81810,
         '3LHet': 8041,
         '3R': 93017,
         '3RHet': 7535,
         '4': 4505,
         'M': 66,
         'X': 74440,
         'XHet': 640,
         'YHet': 778})

In [15]:
grouped = data.groupby('Chromosome')


In [18]:
for category,group in grouped:
    print(category)
    print#(group)
    print(Counter(group['X']))

2L
Counter({False: 76704})
2LHet
Counter({False: 992})
2R
Counter({False: 70488})
2RHet
Counter({False: 9048})
3L
Counter({False: 81810})
3LHet
Counter({False: 8041})
3R
Counter({False: 93017})
3RHet
Counter({False: 7535})
4
Counter({False: 4505})
M
Counter({False: 66})
X
Counter({True: 74440})
XHet
Counter({True: 640})
YHet
Counter({False: 778})
